In [28]:
import defLevel as dlv
import isolation as iso

In [29]:
import pymssql
from sqlalchemy import create_engine 
engine = create_engine("mssql+pymssql://CENTALINE\zhangyun29:sh.8888@./invest")
cnx = engine.connect()

In [30]:
import pandas as pd

In [31]:
df = pd.DataFrame()
# df.columns = ['Date','Tm','O','H','L','C','V']

In [32]:
symbol = '002310'
form_name = 'M1_' + symbol
df = pd.read_sql(form_name,cnx)

In [33]:
dtCnt = len(df)

In [34]:
df.columns = ['Tm','O','H','L','C','V','amount']
df.index = range(len(df))


#===========================Loop K initial=====================================

In [35]:
stdA = [[df.O[0],df.H[0],df.L[0],df.C[0],df.Tm[0]]]
drt_std = 0

Lv1A = [[df.Tm[0],df.O[0], 0]]
cnt_up_Lv1 = 2
cnt_dn_Lv1 = 2

In [36]:
mark = []
Fstd = 0
FLv1 = 0
FLv2 = 0

In [37]:
for i in range(1,dtCnt):
    #print '============================='    
  
    currentK = [df.O[i],df.H[i],df.L[i],df.C[i],df.Tm[i]]
    
    stdA,drt_std,Fstd = dlv.std(currentK,stdA,drt_std)
    istd = len(stdA) -1

    
    if istd > 3 and Fstd == 1:
        Lv1A,FLv1,cnt_up_Lv1,cnt_dn_Lv1 = dlv.Lv1(stdA,Lv1A,cnt_up_Lv1,cnt_dn_Lv1)
    iLv1 = len(Lv1A) - 1
    if (FLv1 == 1 or FLv1 == -1) and iLv1 > 2:
        mark.append([i,Lv1A[iLv1][1],'lv1'])  
        rp1,rp2 = dlv.get_rp(Lv1A,iLv1-3)
        txt = '{:.2}'.format(rp1) +','+ '{:.2}'.format(rp2)
        mark.append([i,Lv1A[iLv1][1]+1,txt])
    
    if iLv1 == 3 and FLv1 == 1 and Fstd == 1:
        
        if Lv1A[2][2] == 1: 
            std_btm =[[Lv1A[0][1],Lv1A[1][1],Lv1A[0][0],0]]
            std_tp =[[Lv1A[2][1],Lv1A[1][1],Lv1A[1][0],0]]            
            
        elif Lv1A[2][2] == -1:
            std_tp =[[Lv1A[1][1],Lv1A[0][1],Lv1A[0][0],0]]
            std_btm =[[Lv1A[1][1],Lv1A[2][1],Lv1A[1][0],0]]
        drt_st = 0
        drt_sb = 0            
                
    #================================Strategy==================================


In [38]:
Lv1A

[[Timestamp('2017-11-22 09:25:00'), 20.620000000000001, 0],
 [Timestamp('2017-11-22 09:35:00'), 20.579999999999998, 1],
 [Timestamp('2017-11-22 09:46:00'), 21.050000000000001, -1],
 [Timestamp('2017-11-22 10:05:00'), 20.789999999999999, 1],
 [Timestamp('2017-11-22 10:12:00'), 20.960000000000001, -1],
 [Timestamp('2017-11-22 10:40:00'), 20.699999999999999, 1],
 [Timestamp('2017-11-22 10:52:00'), 20.989999999999998, -1],
 [Timestamp('2017-11-22 11:20:00'), 20.66, 1],
 [Timestamp('2017-11-22 13:07:00'), 20.780000000000001, -1],
 [Timestamp('2017-11-22 13:49:00'), 20.5, 1],
 [Timestamp('2017-11-22 13:55:00'), 20.59, -1],
 [Timestamp('2017-11-22 14:13:00'), 20.43, 1],
 [Timestamp('2017-11-22 14:20:00'), 20.550000000000001, -1],
 [Timestamp('2017-11-22 14:27:00'), 20.5, 1],
 [Timestamp('2017-11-22 14:31:00'), 20.59, -1],
 [Timestamp('2017-11-22 14:42:00'), 20.359999999999999, 1],
 [Timestamp('2017-11-22 14:52:00'), 20.5, -1]]

In [54]:
names = ['Tm','V','drt']
values = [[1,1,1],[2,2,2],[3,3,3]]
a2 = zip(*Lv1A)
nvs = zip(names,a2)
nvDict = dict( (name,value) for name,value in nvs)
tmp = df.Tm[0]
i = nvDict['Tm'].index(tmp)
len(nvDict)


3

In [20]:
T_count_list = []
for i in range(len(Lv1A)):
    if i < len(Lv1A) - 1:
        t = iso.T_count(Lv1A[i][0],Lv1A[i+1][0],df)
        T_count_list.append(t)
    else:
        T_count_list.append(0)
T_count_list

[6, 11, 19, 7, 28, 12, 28, 18, 42, 6, 18, 7, 7, 4, 11, 10, 0]

In [ ]:
# diff1 = df[df.Tm == df.Tm[3]].index
# diff1
#    Int64Index([3], dtype='int64')

In [23]:
change_list = []
for i in range(len(Lv1A)):
    if i < len(Lv1A) - 1:
        t = abs(Lv1A[i+1][1] - Lv1A[i][1])
        change_list.append(t)
    else:
        change_list.append(0)
change_list

[0.0400000000000027,
 0.47000000000000242,
 0.26000000000000156,
 0.17000000000000171,
 0.26000000000000156,
 0.28999999999999915,
 0.32999999999999829,
 0.12000000000000099,
 0.28000000000000114,
 0.089999999999999858,
 0.16000000000000014,
 0.12000000000000099,
 0.050000000000000711,
 0.089999999999999858,
 0.23000000000000043,
 0.14000000000000057,
 0]

In [24]:
# 求 eva dev max min ....

In [29]:
k_list = []
for i in range(len(T_count_list)-1):
    tmp = change_list[i] / T_count_list[i]
    k_list.append(tmp)
k_list

[0.0066666666666671164,
 0.042727272727272947,
 0.013684210526315872,
 0.024285714285714528,
 0.0092857142857143415,
 0.024166666666666597,
 0.011785714285714224,
 0.0066666666666667217,
 0.006666666666666694,
 0.014999999999999977,
 0.0088888888888888976,
 0.017142857142857286,
 0.0071428571428572441,
 0.022499999999999964,
 0.020909090909090947,
 0.014000000000000058]

In [18]:
from collections import defaultdict
handlers = defaultdict(list)
# 这里的__handlers是一个字典，用来保存对应的事件调用关系
# 其中每个键对应的值是一个列表，列表中保存了对该事件进行监听的函数功能

EVENT_K = 'newK'  

In [19]:
handlerList = handlers[EVENT_K]
import dtDownload as dtd

In [24]:

if dtd.dl_Tick not in handlerList:
    handlerList.append(dtd.dl_Tick)

In [21]:
handlerList

[<function dtDownload.gen_D>]

In [22]:
handlers[EVENT_K]

[<function dtDownload.gen_D>]

In [58]:
handlers

defaultdict(list,
            {0: [],
             1: [],
             3: [],
             'newK': [<function dtDownload.gen_D>,
              <function dtDownload.dl_Tick>]})

In [27]:
d = Dict()        
d['logTime'] = {'chinese':u'时间', 'cellType':BasicCell}
d['logContent'] = {'chinese':u'内容', 'cellType':BasicCell}
d['gatewayName'] = {'chinese':u'接口', 'cellType':BasicCell}

NameError: name 'Dict' is not defined